# Setting up Airflow 2

### Introduction

In this lesson, we'll work with setting up airflow.  Let's get started.

### Setting up With Docker

The easiest way for us to get started with airflow is via Docker.  Before doing so, let's first create a folder where we can place our relevant files, and cd into that directory.

```bash
mkdir airflow-docker
cd airflow-docker
```

From there, we can download the relevant Docker compose file located [here](https://airflow.apache.org/docs/apache-airflow/stable/docker-compose.yaml).  And we can download this by issuig the following command from the `airflow-docker` folder.

```bash
curl -LfO 'https://airflow.apache.org/docs/apache-airflow/2.0.1/docker-compose.yaml'
```

Afterwards, we should see the `docker-compose.yaml` file in our `airflow-docker` directory.

> If you have trouble downloading the file using curl, you can always go to the [file](https://airflow.apache.org/docs/apache-airflow/stable/docker-compose.yaml) and then copy and paste the contents into a `docker-compose.yaml` file.


### Exploring the docker-compose file

The docker-compose file can be really intimidating.  But for now the most important thing to focus on is the component that says `volumes` -- which you can see below.

<img src="./docker-compose-file.png" width="60%">

**First**, look for the line that says `AIRFLOW__CORE__LOAD_EXAMPLES`.  Change the value `'true'` to `'false'`.  Changing this will remove some default examples that airflow comes with, and so removing it will remove some clutter.

Here, in the first line down, docker will take any files in our local `dags` folder and move it into the docker container in the folder `opt/airflow/dags`.  It will do the same with logs, and plugins.  The point is that we should make these folders in our local directory.

<img src="./smaller-mkdirs.png" width="50%">

From there, we can boot up airflow by running the following.

```bash
docker-compose up
```

It may take a couple minutes for all of the related docker containers (aka services) to boot up.  But eventually, you should be able to visit airflow by going to `localhost:8080`.

> We can see that the webserver is mapped to port 8080 in the `docker-compose.yaml` file.

<img src="./airflow-server.png" width="70%">

> If you want to check if all containers are are running, navigate the `airflow-docker` directory, and then type in `docker-compose ps`.

Upon visiting airflow, login with the following credentials:
    

* username: `airflow`
* password: `airflow`

<img src="./airflow-80.png" width="60%">

### Exploring the webserver

From here, we can look under the hood at the webserver for a bit.  We can see that the service `airflow-webserver` has the container name `airflow-docker-airflow-webserver-1`.  

<img src="./docker-services.png" width="70%">

And that means that we can sh into the webserver with the following.

```bash
docker exec -it airflow-docker-airflow-webserver-1 bash
```

Then if we dig around, we can see that under the `/opt/airflow` folder we have our dags, logs and plugins folders.

<img src="./in-container.png" width="80%">

### Adding a Dag

Now let's add a dag to airflow.  We have already added the code to create our first dag in the `/dags/hello_dag.py` file in the `dags` folder of this reading.

This is what it looks like.

```python
# /dags/hello_dag.py
from airflow import DAG
from airflow.decorators import task, dag
from airflow.utils.dates import days_ago
from airflow.utils.task_group import TaskGroup

default_args = {'start_date': days_ago(1)}


@task
def hello_task():
    print('hello world')
    return 'hello world'

@dag(schedule_interval='@once', default_args=default_args, catchup=False)
def hello_world():
    data = hello_task()

dag = hello_world()
```

We'll get into the details of the code later, but if we restart airflow with `docker-compose down` followed by `docker-compose up`, we should see the dag in our airflow website.

This time when we `bash` into our container we can see our `dags/hello_dag.py` file in our container.

<img src="./hello_dags.png" width="100%">

And now we hopefully can see this dag popup if we revisit our airflow webserver by going to `localhost:8080`.

There it is.

> <img src="./hello_w_dag.png" width="90%">

So we can see that our `hello_world` dag was uploaded.  And if we click on that `hello_world` link, then we can see that this dag consists of our `hello_task`.

<img src="./dag-task.png" width="60%">

Now let's try to kick off this dag.  We can do so by going back to the main airflow dashboard, flipping the switch to the left to `unpause` the dag, and then clicking the play button over to the right that says `trigger dag` when hovered over.

<img src="./trigger_dag.png" width="100%">

Then press the trigger button.

<img src="./trigger.png" width="60%">

Then click on the green circle that says `success`.  

<img src="./runs.png" width="70%">

Then click on the Dag run.

<img src="./dag_run.png" width="90%">

And then click on the `hello_task`.

> <img src="./graph_view.png" width="40%">

When we click on the hello_task, we can then click on the log.

<img src="./view-log_2.png" width="60%">

When clicking on the button, we can indeed see the log of task being run.

> <img src="./printed_hello.png" width="100%">

Looking at the log above, we can see that we first see `Marking task as SUCCESS`.  And notice that right above that line, it says 

`hello world`

Remember that this was the return value of the function associated with our task.

```python
def hello():
    return 'Hello world!'

hello_dag = DAG('hello_world', start_date=datetime(2021, 1, 1))

hello_task = PythonOperator(task_id='hello_task', python_callable=hello, dag=hello_dag)
```

So it looks like we were able to create a dag associated with the `hello_task`, and that the `hello_task` then called `hello` which printed out `hello world` function.  

We'll go into more details about the various components of getting this to work in the following lessons, but this is a good place to stop for now.

### Summary

In this lesson, we saw how we can get up and running with airflow by using docker.  We did booted up our airflow container with the command:


`docker-compose up`

And then we created our first dag by placing it into our `/dags` which was then mounted into our container:

From there, we saw that our dag was uploaded to airflow.

> <img src="./hello_world.png" width="80%">

And then we can manually trigger the dag -- by clicking on the play button.

### Resources

[Airflow Setup](https://predictivehacks.com/how-to-start-running-apache-airflow-in-docker/)

[Debugging Airflow](https://www.astronomer.io/blog/7-common-errors-to-check-when-debugging-airflow-dag)